# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("weatherpy.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temperature,Max Temp,Wind Speed
0,Rikitea,94,PF,1580788956,77,-23.12,-134.97,78.84,78.84,11.99
1,Ushuaia,75,AR,1580788956,66,-54.80,-68.30,51.80,51.80,16.11
2,Pasan,38,IN,1580788956,45,22.85,82.20,58.91,58.91,5.91
3,Bredasdorp,92,ZA,1580788956,88,-34.53,20.04,73.90,75.00,5.82
4,Busselton,40,AU,1580788752,19,-33.65,115.33,86.59,90.00,9.42
...,...,...,...,...,...,...,...,...,...,...
546,Kholm,100,RU,1580789051,88,59.27,32.85,24.78,24.78,14.27
547,Iqaluit,90,CA,1580789052,77,63.75,-68.51,5.00,5.00,3.36
548,Walvis Bay,33,NaN,1580789052,93,-22.96,14.51,63.00,63.00,5.99
549,Cherdyn,100,RU,1580789052,97,60.40,56.48,28.00,28.00,6.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]

In [4]:
weather_data["Humidity"].max()

100

In [5]:
fig = gmaps.figure(map_type="TERRAIN")

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
hotel_df = weather_data.loc[(weather_data["Max Temp"] < 80) & (weather_data["Max Temp"] > 70) & (weather_data["Wind Speed"] < 10) & (weather_data["Cloudiness"] == 0)]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temperature,Max Temp,Wind Speed
163,Thayetmyo,0,MM,1580788773,26,19.32,95.18,74.08,74.08,4.61
203,Grand-Santi,0,GF,1580788991,98,4.25,-54.38,70.45,70.45,3.18
246,Aguimes,0,ES,1580788999,60,27.91,-15.45,68.43,73.99,2.24
280,Myanaung,0,MM,1580789005,32,18.28,95.32,71.42,71.42,8.72
281,Tabou,0,CI,1580789005,93,4.42,-7.35,75.33,75.33,3.78
307,Presidencia Roque Saenz Pena,0,AR,1580789009,43,-26.79,-60.44,79.70,79.70,7.00
338,Bonthe,0,SL,1580789014,90,7.53,-12.51,76.26,76.26,6.31
342,Esim,0,GH,1580789015,95,4.87,-2.24,75.20,75.20,1.81
370,Neuquen,0,AR,1580789008,40,-38.95,-68.06,71.60,71.60,2.24
431,Medak,0,IN,1580789031,65,18.03,78.27,75.65,77.00,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
params = {
    "radius": 5000,
    "keyword": "lodging",
    "key": g_key
}

In [8]:
index = 0
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    narrowed_city = requests.get(base_url, params=params)
    #print(narrowed_city.url)
    narrowed_city = narrowed_city.json()
   
    try:
        hotel_df.loc[index, "Hotel Name"] = narrowed_city["results"][0]["name"]
        print(f"Processing Record {index}")
        print(narrowed_city["results"][0]["name"])
        index += 1
    except:
        print("skip")
        pass


C:\Users\ozzmo\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\ozzmo\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Processing Record 163
Aung Lan Hospital
Processing Record 203
Hôtel La Villa Couacou
Processing Record 246
Finca La Manigua
Processing Record 280
Yuzuna Guest House
Processing Record 281
hotêl doufoulougou
Processing Record 307
Hotel Gualok
Processing Record 338
Bonthe Holiday Village
Processing Record 342
Axim Beach Hotel
Processing Record 370
Hostel & Coworking Punto Patagonico
Processing Record 431
Haritha Heritage Hotel
skip


In [9]:
hotel_df = hotel_df.dropna()
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Temperature,Max Temp,Wind Speed,Hotel Name
163,Thayetmyo,0,MM,1580788773,26,19.32,95.18,74.08,74.08,4.61,Aung Lan Hospital
203,Grand-Santi,0,GF,1580788991,98,4.25,-54.38,70.45,70.45,3.18,Hôtel La Villa Couacou
246,Aguimes,0,ES,1580788999,60,27.91,-15.45,68.43,73.99,2.24,Finca La Manigua
280,Myanaung,0,MM,1580789005,32,18.28,95.32,71.42,71.42,8.72,Yuzuna Guest House
281,Tabou,0,CI,1580789005,93,4.42,-7.35,75.33,75.33,3.78,hotêl doufoulougou
307,Presidencia Roque Saenz Pena,0,AR,1580789009,43,-26.79,-60.44,79.70,79.70,7.00,Hotel Gualok
338,Bonthe,0,SL,1580789014,90,7.53,-12.51,76.26,76.26,6.31,Bonthe Holiday Village
342,Esim,0,GH,1580789015,95,4.87,-2.24,75.20,75.20,1.81,Axim Beach Hotel
370,Neuquen,0,AR,1580789008,40,-38.95,-68.06,71.60,71.60,2.24,Hostel & Coworking Punto Patagonico
431,Medak,0,IN,1580789031,65,18.03,78.27,75.65,77.00,3.36,Haritha Heritage Hotel


In [10]:
narrowed_city_df = hotel_df[["Hotel Name", "City", "Country"]]
narrowed_city_df

,Hotel Name,City,Country
163,Aung Lan Hospital,Thayetmyo,MM
203,Hôtel La Villa Couacou,Grand-Santi,GF
246,Finca La Manigua,Aguimes,ES
280,Yuzuna Guest House,Myanaung,MM
281,hotêl doufoulougou,Tabou,CI
307,Hotel Gualok,Presidencia Roque Saenz Pena,AR
338,Bonthe Holiday Village,Bonthe,SL
342,Axim Beach Hotel,Esim,GH
370,Hostel & Coworking Punto Patagonico,Neuquen,AR
431,Haritha Heritage Hotel,Medak,IN


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
#hotel_info

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info, hover_text="Hotel")
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))